# Build a Simple LLM Application with LCEL


In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Using Language Model

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Bahasa Indonesia"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Halo!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd1d6e87-255a-46b0-857b-3745da9e8230-0', usage_metadata={'input_tokens': 22, 'output_tokens': 3, 'total_tokens': 25})

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [8]:
result = model.invoke(messages)

In [9]:
parser.invoke(result)

'Halo!'

In [10]:
chain = model | parser

In [11]:
chain.invoke(messages)

'Halo!'

## Prompt Templates

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
system_template = "Translate the following into {language}, and followed up with a short romantic poem:"

In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [15]:
result = prompt_template.invoke({"language": "indonesian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into indonesian, and followed up with a short romantic poem:'), HumanMessage(content='hi')])

In [16]:
result.to_messages()

[SystemMessage(content='Translate the following into indonesian, and followed up with a short romantic poem:'),
 HumanMessage(content='hi')]

## Chaining Together Componensts with LCEL

In [17]:
chain = prompt_template | model | parser

In [18]:
chain.invoke({"language": "indonesian", "text": "hi"})

'Halo\n\nDi bawah sinar bulan purnama,\nKita berdua berjalan beriringan,\nCinta kita bagai bintang di langit,\nSelalu bersinar terang dalam hati.'

# LangServe

```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

In [19]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao'